In [14]:
import os
from typing import Optional

class InitFolders:
    def __init__(self, folders):
        self.folders = folders

    @staticmethod
    def _create_folder(name: str):

        return os.makedirs(name)

    def init_folders(self, BASE_PATH: Optional[str] = None):
        
        if BASE_PATH is not None:
            for folder in self.folders:
                if folder not in os.listdir(BASE_PATH):
                    self._create_folder(folder)
        else:
            for folder in self.folders:
                if folder not in os.listdir("./"):
                    self._create_folder(folder)

    
